几个超参数
每次测试迭代的次数，

In [4]:
import caffe
import numpy as np
from numpy import array
from numpy.random import normal
from matplotlib import pyplot
import os, sys
import struct

caffe.set_device(0)  
caffe.set_mode_gpu()
prototxt = './VGG16.prototxt'
caffemodel = './VGG16.caffemodel'

In [5]:
def test_accuracy(net, test_iter):
    loss = np.zeros(1)
    top1 = np.zeros(1)
    top5 = np.zeros(1)
    test_loss = 0
    top1_accuracy = 0
    top5_accuracy = 0
    for test_it in range(test_iter):
        net.forward()
        test_loss += net.blobs['loss'].data
        top1_accuracy += net.blobs['accuracy/top-1'].data  
        top5_accuracy += net.blobs['accuracy/top-5'].data
    loss[0] = test_loss / test_iter 
    top1[0] = top1_accuracy / test_iter
    top5[0] = top5_accuracy / test_iter
    print 'Loss: ', loss[0], 'Accuracy Top1:', top1[0], 'Accuracy Top5:', top5[0]
    return loss[0], top1[0], top5[0]

In [6]:
VGG16 = caffe.Net(prototxt, caffemodel, caffe.TEST)
test_accuracy(VGG16, 300)

Loss:  7.288995583852132 Accuracy Top1: 0.0005208333333333333 Accuracy Top5: 0.005208333333333333


(7.288995583852132, 0.0005208333333333333, 0.005208333333333333)

In [22]:
def prune_fm(layer, fm_cnt, iteration):#剪枝哪一层，这一层有几个fm，每次简直测试跑多少iteration
    zeros = np.zeros((3,7,7))
    loss = np.zeros(fm_cnt)
    top1 = np.zeros(fm_cnt)
    top5 = np.zeros(fm_cnt)
    for i in range(fm_cnt):
        googlenet = caffe.Net(prototxt, caffemodel, caffe.TEST)
        googlenet.params[layer][0].data[i]=zeros
        print 'pruning', layer, 'feature map',i, '...'
        loss[i],top1[i],top5[i]=test_accuracy(googlenet, iteration)
    return loss,top1,top5

In [30]:
loss = np.zeros(64)
top1 = np.zeros(64)
top5 = np.zeros(64)
[loss,top1,top5]=prune_fm('conv1_1',64, 1000)

pruning conv1/7x7_s2 feature map 0 ...
Loss:  1.3170900251567363 Accuracy Top1: 0.67296875 Accuracy Top5: 0.8823125
pruning conv1/7x7_s2 feature map 1 ...
Loss:  1.2994334181547165 Accuracy Top1: 0.67528125 Accuracy Top5: 0.88471875
pruning conv1/7x7_s2 feature map 2 ...
Loss:  1.3166984856426716 Accuracy Top1: 0.67340625 Accuracy Top5: 0.8820625
pruning conv1/7x7_s2 feature map 3 ...
Loss:  1.4589048379063607 Accuracy Top1: 0.643375 Accuracy Top5: 0.86490625
pruning conv1/7x7_s2 feature map 4 ...
Loss:  1.3264945199787617 Accuracy Top1: 0.67 Accuracy Top5: 0.8818125
pruning conv1/7x7_s2 feature map 5 ...
Loss:  1.3057714874446391 Accuracy Top1: 0.67625 Accuracy Top5: 0.88353125
pruning conv1/7x7_s2 feature map 6 ...
Loss:  1.3345721195042133 Accuracy Top1: 0.66925 Accuracy Top5: 0.8794375
pruning conv1/7x7_s2 feature map 7 ...
Loss:  1.3539231009483337 Accuracy Top1: 0.6653125 Accuracy Top5: 0.877125
pruning conv1/7x7_s2 feature map 8 ...
Loss:  1.3115352599918841 Accuracy Top1: 0.674

In [25]:
def test_pruned_fm(layer, fm_cnt, prune_cnt, index):
    zeros=np.zeros((3,7,7))
    googlenet = caffe.Net(prototxt, caffemodel, caffe.TEST)
    for i in range(prune_cnt):
        print index[i]
        googlenet.params[layer][0].data[index[i]]=zeros
    test_accuracy(googlenet, 300)

In [29]:
loss_index = np.argsort(loss)#重要程度越低越靠前
top1_index = np.argsort(-top1)
top5_index = np.argsort(-top5)
print 'prune basing on top1...'
test_pruned_fm('conv1/7x7_s2',64,12,top1_index)#砍掉的是重要程度低的fm
print 'prune basing on loss...'
test_pruned_fm('conv1/7x7_s2',64,12,loss_index)

prune basing on top1...
49
47
22
5
12
30
20
60
44
27
19
35
Loss:  1.6941449399789175 Accuracy Top1: 0.5951041666666667 Accuracy Top5: 0.8291666666666667
prune basing on loss...
51
59
24
55
37
15
60
47
44
12
10
28
Loss:  1.6417197988430658 Accuracy Top1: 0.5989583333333334 Accuracy Top5: 0.8383333333333334


In [28]:
loss_index = np.argsort(-loss)# 重要程度越高越靠前
top1_index = np.argsort(top1)
top5_index = np.argsort(top5)
print 'prune basing on top1...'
test_pruned_fm('conv1/7x7_s2',64,24,top1_index)#砍掉的是重要程度高的fm
print 'prune basing on loss...'
test_pruned_fm('conv1/7x7_s2',64,24,loss_index)

prune basing on top1...
3
33
39
21
23
14
57
18
6
16
17
45
36
32
8
58
63
40
53
7
54
4
48
0
Loss:  6.9590077288945515 Accuracy Top1: 0.10322916666666666 Accuracy Top5: 0.22677083333333334
prune basing on loss...
3
33
32
21
23
39
14
57
58
17
54
36
7
53
18
45
40
16
13
62
9
4
6
63
Loss:  6.029995029767354 Accuracy Top1: 0.13135416666666666 Accuracy Top5: 0.27708333333333335
